In [1]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
sys.path.append('../input/pytorch-optimizers/')

In [2]:
import os
import gc
import cv2
import copy
import time
import yaml
import random
import shutil
import pickle
import warnings
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
from PIL import Image, ImageDraw
from shutil import copyfile
from IPython.core.display import Video, display
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold
from sklearn.linear_model import LinearRegression, Ridge

import timm
import torch
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from fastai.vision.all import *

from albumentations.pytorch import ToTensorV2
from albumentations import Resize as albResize
from albumentations import (
    Compose, OneOf, Transpose, RandomResizedCrop, CenterCrop, 
    Rotate, RandomRotate90, ShiftScaleRotate, Flip, HorizontalFlip, VerticalFlip,
    HueSaturationValue, RandomBrightnessContrast, RGBShift, ChannelShuffle,
    Normalize, Blur, MotionBlur, Cutout, CoarseDropout)

warnings.simplefilter('ignore')
pd.set_option("max_columns", 150)
pd.set_option('display.max_rows', 150)

# Configuration

In [3]:
CFG = {
    "submit"      : True,
    "stacking"    : False,
    "seed"        : 42,
    'device'      : "cuda:0" if torch.cuda.is_available() else "cpu",
    "input_trimg" : '../input/petfinder-pawpularity-score/train/',
    "input_trpath": '../input/petfinder-smogn-dataset/train_drop_duplicated.csv',
    "input_teimg" : '../input/petfinder-pawpularity-score/test/',
    "input_tepath": '../input/petfinder-pawpularity-score/test.csv',
    "output_path" : './',
    "db_model"    : "swin_large_patch4_window7_224_in22k",
    "db_size"     : 224,
    "models"      : [[False, 224, 999, 1,  "swin_large_patch4_window7_224_in22k", "../input/petfinder-cnn-models-with-pretrained-edata/petfinder_swin_binary_ss2_meta_0.pt"],
                     [False, 224, 999, 1,  "swin_large_patch4_window7_224_in22k", "../input/petfinder-cnn-models-with-pretrained-edata/petfinder_swin_binary_ss2_meta_1.pt"],
                     [False, 224, 999, 1,  "swin_large_patch4_window7_224_in22k", "../input/petfinder-cnn-models-with-pretrained-edata/petfinder_swin_binary_ss2_meta_2.pt"],
                     [False, 224, 999, 1,  "swin_large_patch4_window7_224_in22k", "../input/petfinder-cnn-models-with-pretrained-edata/petfinder_swin_binary_ss2_meta_3.pt"],
                     [True,  512,   0, 1,                "tf_efficientnet_b5_ns", "../input/petfinder-fastai-models-pseudolabel-1/petfinder_effnet_binary_fastai_ss_0.pth"],
                     [True,  512,   0, 1,                "tf_efficientnet_b5_ns", "../input/petfinder-fastai-models-pseudolabel-1/petfinder_effnet_binary_fastai_ss_1.pth"],
                     [True,  512,   0, 1,                "tf_efficientnet_b5_ns", "../input/petfinder-fastai-models-pseudolabel-1/petfinder_effnet_binary_fastai_ss_2.pth"],
                     [True,  512,   0, 1,                "tf_efficientnet_b5_ns", "../input/petfinder-fastai-models-pseudolabel-1/petfinder_effnet_binary_fastai_ss_3.pth"],
                     [True,  384,   0, 1,                "beit_base_patch16_384", "../input/petfinder-fastai-models-pseudolabel-5/petfinder_beit_binary_fastai_ss_0.pth"],
                     [True,  384,   0, 1,                "beit_base_patch16_384", "../input/petfinder-fastai-models-pseudolabel-5/petfinder_beit_binary_fastai_ss_1.pth"],
                     [True,  384,   0, 1,                "beit_base_patch16_384", "../input/petfinder-fastai-models-pseudolabel-5/petfinder_beit_binary_fastai_ss_2.pth"],
                     [True,  384,   0, 1,                "beit_base_patch16_384", "../input/petfinder-fastai-models-pseudolabel-5/petfinder_beit_binary_fastai_ss_3.pth"],
                     [True,  224,   0, 1,                "vit_large_patch16_224", "../input/petfinder-fastai-models-pseudolabel-4/petfinder_vit_binary_fastai_mixup_0.pth"],
                     [True,  224,   0, 1,                "vit_large_patch16_224", "../input/petfinder-fastai-models-pseudolabel-4/petfinder_vit_binary_fastai_mixup_1.pth"],
                     [True,  224,   0, 1,                "vit_large_patch16_224", "../input/petfinder-fastai-models-pseudolabel-4/petfinder_vit_binary_fastai_mixup_2.pth"],
                     [True,  224,   0, 1,                "vit_large_patch16_224", "../input/petfinder-fastai-models-pseudolabel-4/petfinder_vit_binary_fastai_mixup_3.pth"],
                     [True,  384,   0, 1,                         "cait_s24_384", "../input/petfinder-fastai-models-pseudolabel-4/petfinder_cait_binary_fastai_mixup_0.pth"],
                     [True,  384,   0, 1,                         "cait_s24_384", "../input/petfinder-fastai-models-pseudolabel-4/petfinder_cait_binary_fastai_mixup_1.pth"],
                     [True,  384,   0, 1,                         "cait_s24_384", "../input/petfinder-fastai-models-pseudolabel-4/petfinder_cait_binary_fastai_mixup_2.pth"],
                     [True,  384,   0, 1,                         "cait_s24_384", "../input/petfinder-fastai-models-pseudolabel-4/petfinder_cait_binary_fastai_mixup_3.pth"],
                     [True,  224,   0, 1,  "swin_large_patch4_window7_224_in22k", "../input/petfinder-fastai-models-pseudolabel-5/petfinder_swin_binary_fastai_highlr_0.pth"],
                     [True,  224,   0, 1,  "swin_large_patch4_window7_224_in22k", "../input/petfinder-fastai-models-pseudolabel-5/petfinder_swin_binary_fastai_highlr_1.pth"],
                     [True,  224,   0, 1,  "swin_large_patch4_window7_224_in22k", "../input/petfinder-fastai-models-pseudolabel-5/petfinder_swin_binary_fastai_highlr_2.pth"],
                     [True,  224,   0, 1,  "swin_large_patch4_window7_224_in22k", "../input/petfinder-fastai-models-pseudolabel-5/petfinder_swin_binary_fastai_highlr_3.pth"],
                     [True,  224,   0, 1,  "swin_large_patch4_window7_224_in22k", "../input/petfinder-fastai-models-pseudolabel-2/petfinder_swin_binary_fastai_ss_0.pth"],
                     [True,  224,   0, 1,  "swin_large_patch4_window7_224_in22k", "../input/petfinder-fastai-models-pseudolabel-2/petfinder_swin_binary_fastai_ss_1.pth"],
                     [True,  224,   0, 1,  "swin_large_patch4_window7_224_in22k", "../input/petfinder-fastai-models-pseudolabel-2/petfinder_swin_binary_fastai_ss_2.pth"],
                     [True,  224,   0, 1,  "swin_large_patch4_window7_224_in22k", "../input/petfinder-fastai-models-pseudolabel-2/petfinder_swin_binary_fastai_ss_3.pth"]],
    "tta"         : 2,  # If set over 2, tta will run
    "batch_size"  : 1,
    "num_workers" : 4
}

print(f"# of models is {int(len(CFG['models'])/4)} with fold 4 and tta {CFG['tta']}.")
print(f"Total of inference will be {len(CFG['models']) * CFG['tta']}.")
CFG

# of models is 7 with fold 4 and tta 2.
Total of inference will be 56.


{'submit': True,
 'stacking': False,
 'seed': 42,
 'device': 'cuda:0',
 'input_trimg': '../input/petfinder-pawpularity-score/train/',
 'input_trpath': '../input/petfinder-smogn-dataset/train_drop_duplicated.csv',
 'input_teimg': '../input/petfinder-pawpularity-score/test/',
 'input_tepath': '../input/petfinder-pawpularity-score/test.csv',
 'output_path': './',
 'db_model': 'swin_large_patch4_window7_224_in22k',
 'db_size': 224,
 'models': [[False,
   224,
   999,
   1,
   'swin_large_patch4_window7_224_in22k',
   '../input/petfinder-cnn-models-with-pretrained-edata/petfinder_swin_binary_ss2_meta_0.pt'],
  [False,
   224,
   999,
   1,
   'swin_large_patch4_window7_224_in22k',
   '../input/petfinder-cnn-models-with-pretrained-edata/petfinder_swin_binary_ss2_meta_1.pt'],
  [False,
   224,
   999,
   1,
   'swin_large_patch4_window7_224_in22k',
   '../input/petfinder-cnn-models-with-pretrained-edata/petfinder_swin_binary_ss2_meta_2.pt'],
  [False,
   224,
   999,
   1,
   'swin_large_patc

In [4]:
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

def my_sigmoid(a):
    return 1 / (1 + np.exp(-a))

def softmax(x):
    max = np.max(x,axis=1,keepdims=True)
    e_x = np.exp(x - max)
    sum = np.sum(e_x,axis=1,keepdims=True)
    return e_x / sum 

def seed_everything(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(CFG["seed"])

# Load data

In [5]:
df_train = pd.read_csv(CFG['input_trpath'])
df_train["path"] = [f"{CFG['input_trimg']}{i}.jpg" for i in df_train.Id]
df_train["Pawclass"] = df_train.Pawpularity / 100

if CFG["submit"]:
    df_test = pd.read_csv(CFG['input_tepath'])
    df_test["path"] = [f"{CFG['input_teimg']}{i}.jpg" for i in df_test.Id]
else:
    df_test = pd.read_csv(CFG['input_trpath'])
    df_test["path"] = [f"{CFG['input_trimg']}{i}.jpg" for i in df_test.Id]

meta_features = [c for c in df_test.columns if c not in ["Id","path", "Pawpularity"]]

print(meta_features)
print(df_test.shape)
df_test.head(2)

['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
(8, 14)


,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,path
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,../input/petfinder-pawpularity-score/test/4128bae22183829d2b5fea10effdb0c3.jpg
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,../input/petfinder-pawpularity-score/test/43a2262d7738e3d420d453815151079e.jpg


In [6]:
df_test.describe()

,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur
count,8.000000,8.00000,8.000000,8.00000,8.000000,8.000000,8.000000,8.000000,8.00000,8.000000,8.000000,8.000000
mean,0.625000,0.25000,0.625000,0.25000,0.375000,0.625000,0.500000,0.625000,0.25000,0.500000,0.625000,0.500000
std,0.517549,0.46291,0.517549,0.46291,0.517549,0.517549,0.534522,0.517549,0.46291,0.534522,0.517549,0.534522
min,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,1.000000,0.00000,1.000000,0.00000,0.000000,1.000000,0.500000,1.000000,0.00000,0.500000,1.000000,0.500000
75%,1.000000,0.25000,1.000000,0.25000,1.000000,1.000000,1.000000,1.000000,0.25000,1.000000,1.000000,1.000000
max,1.000000,1.00000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000


# Make dog and cat labels

In [7]:
def load_model(path):
    try:
        checkpoint = torch.load(path, map_location='cpu')
    except Exception as err:
        print(err)
        return None
    model = models.densenet121(pretrained=False)
    model.classifier = nn.Sequential(
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(256, 2)
    )
    model.parameters = checkpoint['parameters']
    model.load_state_dict(checkpoint['state_dict'])
    return model

In [8]:
def image_transform(imagepath):
    test_transforms = transforms.Compose([transforms.Resize(255),
                                          transforms.CenterCrop(224),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])])
    image = Image.open(imagepath)
    imagetensor = test_transforms(image)
    return imagetensor

class PetFinderDataset(Dataset):
    def __init__(self, df):
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        img = image_transform(self.df.loc[index].path)
        return img

In [9]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    img_preds_all = []
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        img_preds = model(imgs)
        img_preds_all += [img_preds.detach().cpu().numpy()]
        
    img_preds_all = np.concatenate(img_preds_all, axis=0)
    return img_preds_all

In [10]:
inference_ds = PetFinderDataset(df_test)
data_loader  = torch.utils.data.DataLoader(inference_ds,
                                           batch_size=CFG['batch_size'],
                                           drop_last=False,
                                           pin_memory=False,
                                           shuffle=False,
                                           num_workers=CFG['num_workers'])
model = load_model("../input/cat-vs-dog-model/cat-v-dog-classifier-pytorch-master/models/catvdog.pth")
model.to(CFG["device"])
with torch.no_grad():
    res_cat_dog = inference_one_epoch(model, data_loader, CFG["device"])

del model, inference_ds, data_loader
torch.cuda.empty_cache()

100%|██████████| 8/8 [00:05<00:00,  1.36it/s]


In [11]:
df_test[["cat","dog"]] = softmax(res_cat_dog)
meta_features += ["cat","dog"]

print(df_test.shape)
df_test.head(2)

(8, 16)


,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,path,cat,dog
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,../input/petfinder-pawpularity-score/test/4128bae22183829d2b5fea10effdb0c3.jpg,0.521388,0.478612
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,../input/petfinder-pawpularity-score/test/43a2262d7738e3d420d453815151079e.jpg,0.576748,0.423252


# Make dog breed for dog label

In [12]:
class SwinModel(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super().__init__()
        self.model   = timm.create_model(model_name, pretrained=pretrained, num_classes=0, in_chans=3)
        num_features = self.model.num_features
        self.linear  = nn.Linear(num_features, 120)

    def forward(self, x):
        x = self.model(x)
        output = self.linear(x)
        return output

In [13]:
class PetFinderDataset(Dataset):
    def __init__(self, df, transforms=None):
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms  = transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        img  = get_img(self.df.loc[index].path).copy()
        if self.transforms:
            img = self.transforms(image=img)['image']
        return img
    
def get_inference_transforms():
    return Compose([
        albResize(CFG["db_size"], CFG["db_size"], p=1.0),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0)], p=1.0)

In [14]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        image_preds = model(imgs)
        image_preds_all += [image_preds.detach().cpu().numpy()]
        
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [15]:
dog_ids = list(df_test[df_test.dog > 0.5].Id)
print(f"Dog num: {len(dog_ids)}")
inference_ds = PetFinderDataset(df_test[df_test.Id.isin(dog_ids)].reset_index(drop=True),
                                        transforms=get_inference_transforms())
data_loader  = torch.utils.data.DataLoader(inference_ds,
                                           batch_size=CFG['batch_size'],
                                           drop_last=False,
                                           pin_memory=False,
                                           shuffle=False,
                                           num_workers=CFG['num_workers'])
model = SwinModel(CFG['db_model'], pretrained=False)
model.load_state_dict(torch.load("../input/petfinder-dogbreed-cnn-models/dogbreed_swin_ce.pt"))
model.to(CFG["device"])
with torch.no_grad():
    res_dogbreed = inference_one_epoch(model, data_loader, CFG["device"])

del model, inference_ds, data_loader
torch.cuda.empty_cache()

Dog num: 2


100%|██████████| 2/2 [00:00<00:00,  8.83it/s]


In [16]:
dogbreed_cols = [f"db{i}" for i in range(res_dogbreed.shape[1])]

df_meta = df_test[meta_features].copy()
df_meta[dogbreed_cols] = 0
df_meta.loc[df_meta.dog > 0.5, dogbreed_cols] = softmax(res_dogbreed)

print(df_meta.shape)
df_meta.head(2)

(8, 134)


,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,cat,dog,db0,db1,db2,db3,db4,db5,db6,db7,db8,db9,db10,db11,db12,db13,db14,db15,db16,db17,db18,db19,db20,db21,db22,db23,db24,db25,db26,db27,db28,db29,db30,db31,db32,db33,db34,db35,db36,db37,db38,db39,db40,db41,db42,db43,db44,db45,db46,db47,db48,db49,db50,db51,db52,db53,db54,db55,db56,db57,db58,db59,db60,db61,db62,db63,db64,db65,db66,db67,db68,db69,db70,db71,db72,db73,db74,db75,db76,db77,db78,db79,db80,db81,db82,db83,db84,db85,db86,db87,db88,db89,db90,db91,db92,db93,db94,db95,db96,db97,db98,db99,db100,db101,db102,db103,db104,db105,db106,db107,db108,db109,db110,db111,db112,db113,db114,db115,db116,db117,db118,db119
0,1,0,1,0,0,1,1,0,0,1,0,1,0.521388,0.478612,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,0,0,0,0,1,1,0,0,0,0,0.576748,0.423252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Define model

In [17]:
class Model(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=3)
        self.n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(self.n_features, 1)

    def forward(self, x):
        output = self.model(x)
        return output
    
class ModelwithMetadata(nn.Module):
    def __init__(self, model_name, size=512, pretrained=True):
        super().__init__()
        self.size  = size
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=4)
        self.n_features = self.model.classifier.in_features
        # Exclude the top layer
        self.model.reset_classifier(0)
        self.linear1 = nn.Linear(134, size*size)        
        self.linear2 = nn.Linear(134, self.n_features)
        self.linear3 = nn.Linear(self.n_features, 256)
        self.linear4 = nn.Linear(256, 1)
        self.relu    = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x1, x2):
        # Metadata first
        x = self.linear1(x2)
        x = torch.reshape(x, (x2.shape[0], 1, self.size, self.size))
        x = torch.cat((x1, x), dim=1)
        x = self.model(x)
        # Metadata Last
        x2 = self.linear2(x2)
        x  = torch.add(x, x2)
        x  = self.relu(self.linear3(x))
        x  = self.dropout(x)
        output = self.linear4(x)
        return output

class TransformerModel(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super().__init__()
        self.model   = timm.create_model(model_name, pretrained=pretrained, num_classes=0, in_chans=3)
        num_features = self.model.num_features
        self.linear  = nn.Linear(num_features, 1)
        
    def forward(self, x):
        x = self.model(x)
        output = self.linear(x)
        return output
    
class TransformerModelwithMetadataLast(nn.Module):
    def __init__(self, model_name, size=224, pretrained=True):
        super().__init__()
        self.size  = size
        self.backbone = TransformerModel(model_name, False)
        num_features  = self.backbone.model.num_features
        self.backbone.linear = nn.Linear(num_features, 256)
        self.linear1 = nn.Linear(134, 256)
        self.linear2 = nn.Linear(256, 128)
        self.linear3 = nn.Linear(128, 1)
        self.relu    = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x1, x2):
        x  = self.backbone(x1)
        # Metadata Last
        x2 = self.linear1(x2)
        x  = torch.add(x, x2)
        x  = self.relu(self.linear2(x))
        x  = self.dropout(x)
        output = self.linear3(x)
        return output

In [18]:
class PetFinderDataset(Dataset):
    def __init__(self, df_img, df_meta, size, transforms=None, output_meta=True):
        super().__init__()
        self.df_img  = df_img.reset_index(drop=True).copy()
        self.df_meta = df_meta.reset_index(drop=True).copy()
        self.size    = size
        self.transforms  = transforms
        self.output_meta = output_meta
        
    def __len__(self):
        return self.df_img.shape[0]
    
    def __getitem__(self, index: int):
        img  = get_img(self.df_img.loc[index].path)
        meta = torch.from_numpy(np.array(self.df_meta.loc[index], dtype=float))
        if self.transforms:
            h, w, _ = img.shape
            trans = self.transforms(self.size, h, w)
            img   = trans(image=img)['image']
        if self.output_meta:
            return img, meta
        return img

In [19]:
def get_tta_inference_transforms_1(size, h, w):
    return Compose([
        RandomResizedCrop(size, size, scale=(0.7, 1.0), p=1.0),
        HorizontalFlip(p=0.5),
        MotionBlur(p=0.5),
        HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
        RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0)], p=1.0)

def get_tta_inference_transforms_2(size, h, w):
    return Compose([
        albResize(size, size, p=1.0),
        HorizontalFlip(p=0.5),
        HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
        RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0)], p=1.0)

def get_inference_transforms_1(size, h, w):
    h = int(size*1.2) if int(size*1.2) < h else h
    w = int(size*1.2) if int(size*1.2) < w else w
    return Compose([
        CenterCrop(h, w, p=1.0),
        albResize(size, size, p=1.0),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0)], p=1.0)

def get_inference_transforms_2(size, h, w):
    return Compose([
        albResize(size, size, p=1.0),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0)], p=1.0)

In [20]:
def inference_one_epoch(model, use_meta, data_loader, device):
    model.eval()

    img_preds_all = []
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs, metas) in pbar:
        imgs  = imgs.to(device).float()
        if use_meta != 0:
            metas = metas[:,:use_meta].to(device).float()
            img_preds = model(imgs, metas)
        else:
            img_preds = model(imgs)
        img_preds_all += [img_preds.detach().cpu().numpy()]
        
    img_preds_all = np.concatenate(img_preds_all, axis=0)
    return img_preds_all

In [21]:
def prepare_dataloader(df, size):
    df = df.copy()
    label_col  = "Pawclass"
    dataloader = ImageDataLoaders.from_df(
        df,
        valid_pct=0.2,  # Dummy
        seed=CFG["seed"],
        fn_col='path',
        label_col=label_col,
        y_block=RegressionBlock,
        bs=CFG['batch_size'],
        num_workers=CFG['num_workers'],
        item_tfms=Resize(size),
        batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])
    )
    return dataloader

def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

In [22]:
def get_learner(df, size, model, model_path):
    dataloader = prepare_dataloader(df, size)
    if -1 < max(model.find("swin"), model.find("beit"), model.find("vit"), model.find("cait")):
        model = TransformerModel(model, pretrained=False)
    else:
        model = Model(model, pretrained=False)
    model.load_state_dict(torch.load(model_path))
    learner = Learner(
        dataloader,
        model,
        loss_func=BCEWithLogitsLossFlat(),
        metrics=petfinder_rmse).to_fp16()
    return learner, dataloader

# Run inference

In [23]:
all_res = []
for fastai, size, use_meta, dataset_type, model, model_path in CFG['models']:
    
    if fastai:
        learn, data_loader = get_learner(df_train, size, model, model_path)
        data_loader = data_loader.test_dl(df_test)
        res, _ = learn.tta(dl=data_loader, n=CFG["tta"], beta=0)
        res    = res.detach().numpy()
        if -1 < model_path.find("binary"):
            res = res*100
        all_res.append(res)
        
        del learn, data_loader
    else:
        if 1 < CFG["tta"]:
            if dataset_type == 1:
                inference_ds = PetFinderDataset(df_test, df_meta, size, transforms=get_tta_inference_transforms_1)
            else:
                inference_ds = PetFinderDataset(df_test, df_meta, size, transforms=get_tta_inference_transforms_2)
        else:
            if dataset_type == 1:
                inference_ds = PetFinderDataset(df_test, df_meta, size, transforms=get_inference_transforms_1)
            else:
                inference_ds = PetFinderDataset(df_test, df_meta, size, transforms=get_inference_transforms_2)
        data_loader = torch.utils.data.DataLoader(inference_ds,
                                                  batch_size=CFG['batch_size'],
                                                  drop_last=False,
                                                  pin_memory=False,
                                                  shuffle=False,
                                                  num_workers=CFG['num_workers'])
        if -1 < model_path.find("swin"):
            if use_meta == 0:
                model = TransformerModel(model, pretrained=False)
            else:
                model = TransformerModelwithMetadataLast(model, size, pretrained=False)
        else:
            if use_meta == 0:
                model = Model(model, pretrained=False)
            else:
                model = ModelwithMetadata(model, size, pretrained=False)
        model.load_state_dict(torch.load(model_path))
        model.to(CFG["device"])

        tta_res = []
        for i in range(CFG["tta"]):
            with torch.no_grad():
                res = inference_one_epoch(model, use_meta, data_loader, CFG["device"])
                if -1 < model_path.find("binary"):
                    res = my_sigmoid(res)*100
                tta_res.append(res)
        all_res.append(np.mean(tta_res, 0))

        del model, inference_ds, data_loader
    torch.cuda.empty_cache()

100%|██████████| 8/8 [00:00<00:00, 24.66it/s]


In [24]:
df_pred = pd.DataFrame(np.array(all_res).reshape(len(CFG["models"]),-1)).T
if not CFG["submit"]:
    df_pred["target"] = df_test.Pawpularity
    df_pred.to_csv("./inference_result_train_data.csv", index=False)

In [25]:
if CFG["stacking"]:
    loaded_model = pickle.load(open("../input/petfinder-stacking-model/stacking_model.pickle", 'rb'))
    # Calculate mean every model type
    sta = 0
    mean_every_model = []
    for i in range(4, len(CFG["models"])+1, 4):  # 4 is fold num
        mean_every_model.append(df_pred.iloc[:, sta:i].mean(1))
        sta = i
    df_mean_every_model = pd.concat(mean_every_model, axis=1)
    # Predict with stacking model
    ensembled = loaded_model.predict(df_mean_every_model)
else:
    ensembled = df_pred.mean(1)
print(ensembled)

0    42.951481
1    43.533123
2    43.402565
3    43.323055
4    43.445732
5    43.033955
6    42.682770
7    43.069897
dtype: float32


In [26]:
if CFG["submit"]:
    ss = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
    ss["Pawpularity"] = ensembled
    ss.to_csv("./submission.csv", index=False)